### Inputs

- **Bonds with maturities**: 1, 2, 3, ... years (to allow sequential bootstrapping)
- **Market price of each bond**: observed traded price in the market
- **Face value** \(N\)
- **Coupon rate** (annual, as a percentage or decimal)
- **Maturity date** of each bond
- **Compounding frequency** \(n\): number of coupon/compounding periods per year


### OUTPUT :
— Nominal rates $R_{T_i}$ corresponding to each cash flow $T_i$

### TO DO
—Find an API that give bond prices
—Find the bonds I want to study (let's take simple first)

### First Choices
- I will use to verify my script : cleanest, most vanilla Treasuries bonds
- US nominal Treasuries (fixed coupon, non-callable, not inflation linked)
- Maturities : 3M, 6M, 1Y

I have created a script that import all bond prices I need from FRED. I can import the data here in the notebook to manipulate it 

First I have imported the zero-coupon bond rates
So this data is the objective : it is for a given date what I should find as zero coupon bonds

In [ ]:
from import_bond_data import fetch_treasury_yields

df, curve_date, yields_today = fetch_treasury_yields()

df.head, curve_date, yields_today

(            DGS3MO  DGS6MO   DGS1
 1981-09-01   17.01   17.17  17.06
 1981-09-02   16.65   17.32  17.16
 1981-09-03   16.96   17.42  17.31
 1981-09-04   16.64   17.37  17.24
 1981-09-08   16.54   17.43  17.29
 ...            ...     ...    ...
 2025-11-19    3.95    3.80   3.68
 2025-11-20    3.94    3.79   3.65
 2025-11-21    3.90    3.75   3.62
 2025-11-24    3.91    3.76   3.61
 2025-11-25    3.90    3.75   3.60
 
 [11058 rows x 3 columns],
 Timestamp('2025-11-25 00:00:00'),
 DGS3MO    3.90
 DGS6MO    3.75
 DGS1      3.60
 Name: 2025-11-25 00:00:00, dtype: float64)

Now I am facing a big issue. 
- I can't have the data of coupon paying bonds as it is often very expensive APIs like Bloomberg, Refinitiv etc.
- Thus I need to change my goal and method.
- I will focus on the code and math

### Choice of Data for Bootstrapping – Why I Use Synthetic Coupon Bonds 

In real markets, the bootstrapping algorithm is applied to **actual traded instruments** (Treasury bills, notes, bonds, swaps) whose **prices and yields come from the market** (e.g. Bloomberg, Refinitiv, broker feeds).  
Those raw quotes are then cleaned and used to construct an arbitrage-free zero-coupon curve.

In this project, I **don’t have direct access** to that kind of professional market-data feed.  
I *do* have FRED data for **3M, 6M, 1Y Treasuries**, which are effectively **zero-coupon (bill) yields**, and I will treat these as a **“true” short-end zero curve** for intuition and comparison.

However, to **implement and test the bootstrapping algorithm itself**, I also need a set of **coupon-paying bonds** (e.g. 2Y, 3Y, 5Y notes) with given coupons and prices.  
Since I can’t easily pull a clean set of real Treasury bond quotes programmatically, I will:

- Construct a small universe of **synthetic Treasury-like coupon bonds** (with realistic coupons and prices) directly in code/CSV.
- **Other options I have but seem difficult for now**:
  - Reconstructing bonds from public par curves (those are already bootstrapped, so I’d be re-bootstrapping a bootstrapped curve).
  - Using a professional data vendor or broker API (too heavy / not accessible for this learning project).

This choice keeps the project **conceptually honest** and **technically focused**:

- I **clearly separate**:
  - The *source of prices* (here: synthetic but realistic coupon bonds),
  - From the *algorithm* (bootstrapping those prices into a zero-coupon curve).
- The synthetic bonds are **not arbitrary**; they are designed to look like real Treasury notes (maturities, coupons, price levels).
- In a real bank, the **same mathematical algorithm** I code here would be used, but the input prices would come from live market-data feeds instead of synthetic data.

So knowing everything mentionned, I first created the yield curve based on FRED rates (sero-coupon rates) which is our basis of comparison.